In [4]:
import pyarrow.parquet as pa
parquet_file = pa.read_table("amazon_fashion_clean_051624.parquet")


In [5]:
import pandas as pd

df = parquet_file.to_pandas()

In [6]:
df.shape

(776976, 18)

In [7]:
df.columns

Index(['title', 'average_rating', 'rating_number', 'features', 'description',
       'price', 'images', 'store', 'details', 'parent_asin',
       'title_review_agg', 'user_id', 'timestamp', 'avg_rating_reviewers',
       'coefvar_rating_reviewers', 'text_agg', 'text_weighted_agg',
       'images_review_cln'],
      dtype='object')

In [21]:
len(df['details'][0])

534

In [19]:
# Normalize the 'details' column into a separate DataFrame
details_df = pd.json_normalize(df['details'])


In [20]:

# Concatenate the 'title' column with the normalized details DataFrame
result_df = pd.concat([df[['title']], details_df], axis=1)

# Display the result
print(result_df)

                                                    title        \
0       CHUVORA 925 Sterling Silver Open Celtic Knot C...  None   
1       XX-Large Slip Stop Single Tread Slipper Socks ...  None   
2       Sterling Silver 3mm Round Cut CZ Tennis Bracel...  None   
3       VERO MONTE 4 Pairs Womens TRULY No Show Socks ...  None   
4       SA106 Womens Rhinestone Jewel Polarized Lens 6...  None   
...                                                   ...   ...   
776971  ForLeatherMore Genuine Leather Lamb Drawing En...  None   
776972  Magnificent Mr. Claus Christmas Suit | Festive...  None   
776973  Workout Longline Padded Yoga Sports Bra Fitnes...  None   
776974  AITFINEISM Classic Slim Fit Bomber Jacket Men ...  None   
776975  DINGANG Lantern Dress for Women Summer Casual ...  None   

       ABPA Partslink Number ASTM Fluid Rating Active Ingredients  \
0                       None              None               None   
1                       None              None           

In [30]:
# Calculate the percentage of missing values in each column
missing_percentage = result_df.isnull().mean() * 100

# Display columns with less than a certain threshold of missing values
threshold = 75 # For example, 50%
significant_columns = missing_percentage[missing_percentage < threshold].index.tolist()
print(f"Columns with less than {threshold}% missing values:", significant_columns)

Columns with less than 75% missing values: ['title', 'Date First Available', 'Is Discontinued By Manufacturer', 'Item model number', 'Package Dimensions']


In [27]:
nonnull_counts = result_df.notnull().sum()

# Aggregate how many columns have x non-null entries
column_count_summary = nonnull_counts.value_counts().sort_index()

# Display the summary
print(column_count_summary)

0           1
1         162
2          53
3          37
4          21
         ... 
285655      1
353521      1
494779      1
749608      1
776976      1
Name: count, Length: 152, dtype: int64


In [35]:
column_count_summary[-1:-100:-1]

776976    1
749608    1
494779    1
353521    1
285655    1
         ..
100       1
99        2
97        1
92        1
87        1
Name: count, Length: 99, dtype: int64